In [1]:
# %% [markdown]
# # 🔬 Predicción de mineralogía sin transformación CLR
# 
# Este notebook evalúa el rendimiento de un modelo Random Forest multisalida para predecir porcentajes de arcillas, cuarzo y carbonatos directamente (sin usar transformaciones composicionales como CLR).
# 
# **Objetivo:** Evaluar si este enfoque mejora la capacidad de generalización, especialmente en valores extremos.

# %% 📦 Librerías
import warnings
warnings.filterwarnings("ignore")

# Manipulación de datos
import numpy as np
import pandas as pd

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Estilo visual
plt.rcParams["figure.dpi"] = 300

In [ ]:
import lasio
import pandas as pd
import glob
import os

# Ruta donde están tus LAS
ruta = "data/Curvas VSH"

# Diccionario para guardar los dataframes
dfs = {}

for file in glob.glob(ruta):
    nombre = os.path.splitext(os.path.basename(file))[0]  # nombre sin extensión
    las = lasio.read(file)
    df = las.df()  # convierte a dataframe con profundidad como índice
    df["DEPT"] = las.index  # opcional: poner la profundidad explícita como columna
    dfs[nombre] = df

# Ejemplo de acceso:
print(dfs["CERRO GORDO-3_PETROFISICA_2025_QC"].head())
